This notebook can be used to create new diamond datasets, new datasets for training the svm, and also for training the svm. It also has code for creating the background spins used in the last step of the code.

In [1]:
%load_ext autoreload
%autoreload 1
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
%aimport analysis
%aimport learning
%aimport NV_generator

In [2]:
B_field = 0.0403555 # Teslas
gam_c = 67.262 * 10 ** 6 # Gyromagnetic ratio for a single c13 nucleus in rad s-1 T-1
ms = 1
omega_larmor = -1 * ms * gam_c * B_field

In [3]:
num_diamonds = 29

In [4]:
if False:
    learning.create_diamonds(range(num_diamonds, 2 * num_diamonds), omega_larmor, num_spins = 450)

In [66]:
diamonds = []
for diamond_num in range(num_diamonds):
    print "loading diamond ", diamond_num
    diamond = learning.load_obj("diamonds/diamond_" + str(diamond_num))
    diamonds.append(diamond)
print "diamonds loaded"
suffix = "_di3220_29diamonds_cxABe"
if False:
    _ = learning.guess_dataset(diamonds, omega_larmor, pickle = True, training_percent = .5, min_dip_ind = 3220, suffix = suffix)

loading diamond  0
loading diamond  1
loading diamond  2
loading diamond  3
loading diamond  4
loading diamond  5
loading diamond  6
loading diamond  7
loading diamond  8
loading diamond  9
loading diamond  10
loading diamond  11
loading diamond  12
loading diamond  13
loading diamond  14
loading diamond  15
loading diamond  16
loading diamond  17
loading diamond  18
loading diamond  19
loading diamond  20
loading diamond  21
loading diamond  22
loading diamond  23
loading diamond  24
loading diamond  25
loading diamond  26
loading diamond  27
loading diamond  28
diamonds loaded
diamond_ind:  0
diamond_ind:  1
diamond_ind:  2
diamond_ind:  3
diamond_ind:  4
diamond_ind:  5
diamond_ind:  6
diamond_ind:  7
diamond_ind:  8
diamond_ind:  9
diamond_ind:  10
diamond_ind:  11
diamond_ind:  12
diamond_ind:  13
diamond_ind:  14
diamond_ind:  15
diamond_ind:  16
diamond_ind:  17
diamond_ind:  18
diamond_ind:  19
diamond_ind:  20
diamond_ind:  21
diamond_ind:  22
diamond_ind:  23
diamond_ind:  24

analysis.py:61: RuntimeWarning: invalid value encountered in sqrt
  B = np.sqrt((1 - mz ** 2)) * omega_tilde


In [6]:
suffix = "_di3220_29diamonds_cxABe"
dataset = learning.load_obj("datasets/guess_dataset" + suffix)
if False:
    print "train svm"
    scaler, clf = learning.train_svm(dataset["train_X"], dataset["train_Y"], dataset["test_X"], dataset["test_Y"],
                                 kernel='linear', verbose = True, pickle = True, suffix = suffix)

train svm
training score:  0.913136857177
testing score:  0.909582799373


In [7]:
clf.coef_, clf.intercept_

(array([[-0.36175828,  0.95493479,  0.00244094,  0.03401229, -0.02430468]]),
 array([-0.94027893]))

In [26]:
A_background = 4 * analysis.mag * (np.random.rand(400) - .5)
B_background = 2 * analysis.mag * (np.random.rand(400))
background_dict = {"A_background" : A_background, "B_background" : B_background}
if False:
    learning.store_obj(background_dict, "background_A_B")